In [1]:
import numpy as np
import pandas as pd

In [2]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2

TEXT_DATA = 'data/fake_or_real_news.csv'

In [3]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

In [4]:
# estimate 95% confidence interval on error

# NOTE: based on conversation on stackexchange: 
# https://stats.stackexchange.com/questions/247551/how-to-determine-the-confidence-of-a-neural-network-prediction
# towards bottom of the page.

from math import sqrt

def error_conf(error, n):
    term = 1.96*sqrt((error*(1-error))/n)
    lb = error - term
    ub = error + term
    
    return lb, ub

In [5]:
# read in our data and preprocess it
# Note: the news items in the data set range from 0 words to over 100,000 words
# we restrict ourselves to news items between 500 and 5,000 words.

df = pd.read_csv(TEXT_DATA)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with between 500 and 5000 words
mask = list(df['text'].apply(lambda x: len(x) >= 500 and len(x) <= 5000))
df = df[mask]

In [6]:
# prepare text samples and their labels                                                                  
texts = df['text']
labels = df['label'].apply(lambda x: 0 if x == 'FAKE' else 1)

print('Found %s texts.' %texts.shape[0])

Found 3510 texts.


In [7]:
# vectorize the text samples into a 2D integer tensor                                                   
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Found %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Using TensorFlow backend.


Found 53510 unique tokens.
Shape of data tensor: (3510, 1000)
Shape of label tensor: (3510,)


In [8]:
# split the data into a training set and a validation set   
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=TEST_SPLIT)

In [9]:
# create our (randomly initialized) embedding layer
from keras.layers import Embedding

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [10]:
# build a 1D convnet with global maxpooling                                                                      
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model

input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
x = embedding_layer(input_layer)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 2808 samples, validate on 702 samples
Epoch 1/10
2808/2808 [==============================] - 22s 8ms/step - loss: 0.6187 - acc: 0.6100 - val_loss: 0.4172 - val_acc: 0.8063
Epoch 2/10
2808/2808 [==============================] - 21s 7ms/step - loss: 0.2825 - acc: 0.9017 - val_loss: 0.1791 - val_acc: 0.9259
Epoch 3/10
2808/2808 [==============================] - 21s 7ms/step - loss: 0.0835 - acc: 0.9736 - val_loss: 0.1554 - val_acc: 0.9316
Epoch 4/10
2808/2808 [==============================] - 21s 7ms/step - loss: 0.0592 - acc: 0.9808 - val_loss: 0.1888 - val_acc: 0.9274
Epoch 5/10
2808/2808 [==============================] - 21s 7ms/step - loss: 0.0068 - acc: 0.9993 - val_loss: 0.1623 - val_acc: 0.9359
Epoch 6/10
2808/2808 [==============================] - 21s 8ms/step - loss: 9.5194e-04 - acc: 1.0000 - val_loss: 0.1745 - val_acc: 0.9359
Epoch 7/10
2808/2808 [==============================] - 21s 7ms/step - loss: 1.2162e-04 - acc: 1.0000 - val_loss: 0.1881 - val_acc: 0.9402


In [12]:
# evaluate model

def predict(X):
    return np.rint(model.predict(X)) # threshold the predictions to retrieve labels

train_acc, test_acc = evaluate_model(predict,
                                     x_train, 
                                     y_train, 
                                     x_val, 
                                     y_val)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 100.00%
Testing Accuracy: 93.87%


In [13]:
# estimating 95% confidence interval
n = x_val.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 92.10%-95.65%
